 奇异值分解（Singular Value Decomposition，SVD）作为一种常用的**矩阵分解和数据降维**方法，在机器学习中也得到了广泛的应用，比如自然语言处理中的SVD词向量和潜在语义索引，推荐系统中的特征分解，SVD用于PCA降维以及图像去噪与压缩等。作为一个基础算法，我们有必要将其单独拎出来在机器学习系列中进行详述。\


**A的矩阵分解**\
 **$n \times n$方阵分解(方阵才能计算特征值和特征向量)**
$$1. 根据 如下直接求出A的特征值和特征向量\\
Ax=\lambda x $$
> 
> $$2. 然后矩阵就可以用下式进行分解（矩阵的对角化/求矩阵的相似矩阵）\\
W 由特征值\lambda_1、\lambda_2....\lambda_n组成 \\
\Lambda 是由对应的特征向量w_1、w_2...w_n组成的对角阵\\
A=W\Lambda W^{-1}$$
> 
> $$3.一般我们会将矩阵W的n个特征向量进行标准化和正交化处理，\\
使得W满足W^TW=E，所以就有W_T=W_{-1}，即W为酉矩阵。\\
最终上述分解表达式可表示为：\\
A=W\Lambda W^T$$ 

大多数情况下，我们碰到的矩阵都是非方阵的

**$m \times n $非方阵分解(SVD)**
> 1. 定义分解表达式为：
> $$A=U\Lambda V^T\\
U:m \times m矩阵， \Lambda : m \times n 对角阵，V: n\times n 矩阵\\
U、V:酉矩阵,故U、V满足：\\
UU^T=E,VV^T=E$$
> 2. 求 左奇异向量 矩阵$U$ \
> $A为非方阵，但AA^T为m\times m 方阵,故对该矩阵求特征值和特征向量：$
$$(AA^T)x=\lambda x \\
由上式我们即可求得方阵AA^T的m个特征值和特征向量，\\
该m个特征向量构成的特征矩阵即为矩阵U,特征值矩阵W
$$
>3. 求 右奇异向量 矩阵$V$ \
$$同2,求A^TA(n\times n)的特征值和特征向量：\\
(A^TA)x=\lambda x\\
该n个特征向量构成的特征矩阵即为矩阵V$$
>4. 求奇异值对角阵 $\Lambda$\
$$A=U\Lambda V^T\\
A^T=V\Lambda U^T\\
AA^T=U\Lambda V^T V\Lambda U^T=U\Lambda^2 U^T\\
故 AA^T的特征值矩阵W= \Lambda^2,由上2可求得W，\Lambda=\sqrt{W} $$

In [18]:
from PIL import Image 
c=Image.open('demo.PNG','r')
A=np.array(c) 
A.shape 
A.dtype 
# A=A.astype('float')
# Image.fromarray(A).show()

dtype('uint8')

In [19]:
# 这个计算出来的和numpy自带函数计算出来有点偏差
def svd(A):
    sigmal_1,U=np.linalg.eigh(A.dot(A.T))
    sigmal_2,V=np.linalg.eigh(A.T.dot(A))
    S=np.sqrt(sigmal_1)
    S.sort()
    return U,S[::-1],V 
# 定义恢复函数，由分解后的矩阵恢复到原矩阵
def restore(u,s,v,K):
    a=np.dot(u[:,:K]*s[:K],v[:K,:])
    return a.clip(0,255).astype('uint8') # uint8格式 取值范围0-255 ,图像格式，其他格式不支持
# 使用前50个奇异值
K=50
# 对RGB图像进行奇异值分解
u_r,s_r,v_r=np.linalg.svd(A[:,:,0])
u_g,s_g,v_g=np.linalg.svd(A[:,:,1]) 
u_b,s_b,v_b=np.linalg.svd(A[:,:,2])  
out_path='svd_image/'
I=None 
for k in range(2,K+1):
    R=restore(u_r,s_r,v_r,k)
    G=restore(u_g,s_g,v_g,k) 
    B=restore(u_b,s_b,v_b,k)
    I=np.stack((R,G,B),axis=2)
    Image.fromarray(np.stack((R,G,B),axis=2)).show()
    break 
    Image.fromarray(np.stack((R,G,B),axis=2)).save(out_path+f"svd_{k}.png")
    print(f'{k} is ok!')
